In [204]:
import pandas as pd
import numpy as np
import json
# import psycopg2
# from sqlalchemy import create_engine
# import os
# from dotenv import load_dotenv
# load_dotenv()

# from script.database_pipeline import Pipeline
from script.helper import *

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('darkgrid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Setting up DB

**Decided to put database on hold**

In [ ]:
# username = os.getenv('PSQL_USER')
# password = os.getenv('PASS')
# host = os.getenv('HOST')
# port = os.getenv('PORT')

# db_url = (f"postgresql+psycopg2://{username}:{password}@{host}:{port}/data_prof")

In [ ]:
# engine = create_engine(db_url)

In [ ]:
# Converted into a csv
df = pd.read_csv('ds_2019.csv')

In [ ]:
# pipe = Pipeline(engine)
# pipe.add_table(df, 'og')

### EDA

Exploring and cleaning

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().mean()

In [ ]:
countries = df['Country'].value_counts()
countries.head()

### Looking at US to start with

since majority of the data is in US

In [ ]:
us = df[df['Country'] == 'United States'].copy()

In [ ]:
us.isna().mean()

In [ ]:
(us == 'Not Asked').mean(axis=0)

In [ ]:
us[us['CareerPlansThisYear'] == 'Not Asked']['Survey Year'].value_counts()

In [ ]:
features = ['SalaryUSD', 'YearsWithThisDatabase', 'ManageStaff', 'YearsWithThisTypeOfJob', 'OtherPeopleOnYourTeam', \
            'DatabaseServers', 'Education', 'HoursWorkedPerWeek', 'TelecommuteDaysPerWeek', 'EmploymentSector', \
            'Gender', 'OtherJobDuties']
target = ['LookingForAnotherJob']

### Salary

In [ ]:
us_cleaned = us.copy()

In [ ]:
us_cleaned.loc[:, 'SalaryUSD'] = us['SalaryUSD'].apply(str).replace(',|\s+', '', regex=True).astype(float)

In [ ]:
us_cleaned['SalaryUSD'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_quant(ax, us_cleaned['SalaryUSD'], xlabel='Salary not adj.', ylabel='Frequency',
                 title='Frequency of Salary, not adj.', shade=True))

### Remove salary outliers

In [ ]:
salary = us_cleaned['SalaryUSD']

In [ ]:
salary.describe()

In [ ]:
salary_q3 = salary.quantile(.75)
salary_q1 = salary.quantile(.25)

salary_iqr = salary_q3 - salary_q1
salary_adj = salary[(salary < (salary_q3 + 1.5 * salary_iqr)) & (salary > (salary_q1 - 1.5 * salary_iqr))]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_quant(ax, salary_adj, xlabel='Salary adj.', ylabel='Frequency',
                 title='Distribution of Salary adj.', shade=True))

### Postal Code

In [658]:
us_cleaned['PostalCode']

0       Not Asked
12      Not Asked
14      Not Asked
15      Not Asked
17      Not Asked
          ...    
6883        44122
6884        29201
6886        03873
6887        50266
6891        49506
Name: PostalCode, Length: 4509, dtype: object

In [450]:
pc = us_cleaned.loc[(us_cleaned['PostalCode'] != 'Not Asked'), 'PostalCode'].values

In [680]:
with open('postcodeclean.json', 'r') as f:
    results_combined = json.load(f)

In [660]:
def map_zipcode(row):
    if row in results_combined:
        return results_combined[row]
    elif row == 'WI':
        return 'Wisconsin'
    elif row == 'DC':
        return 'District of Columbia'
    return row

In [668]:
us_cleaned.loc[:, 'PostalCodeStates'] = us['PostalCode'].apply(map_zipcode)

In [682]:
us_cleaned['PostalCodeStates'].value_counts()

Not Asked               1879
California               208
Texas                    194
Washington               122
Illinois                 122
Pennsylvania             114
Florida                  106
Ohio                      98
New York                  85
Colorado                  85
Minnesota                 85
Wisconsin                 76
Georgia                   75
North Carolina            74
Missouri                  69
Michigan                  60
Arizona                   59
Oregon                    57
Tennessee                 53
Indiana                   52
Virginia                  51
Massachusetts             49
Utah                      41
Maryland                  36
Kansas                    35
Iowa                      32
New Jersey                27
South Carolina            25
Oklahoma                  24
Kentucky                  21
Idaho                     18
New Hampshire             17
Connecticut               17
Arkansas                  16
Nebraska      

### Years with this database

In [ ]:
us['YearsWithThisDatabase'].value_counts().sort_index()

In [ ]:
us_cleaned.loc[:, 'YearsWithThisDatabase'] = us.loc[us['YearsWithThisDatabase'] < 100, 'YearsWithThisDatabase']

In [ ]:
year_db = us_cleaned['YearsWithThisDatabase']

In [ ]:
yeardb_q3 = year_db.describe()['75%']
yeardb_q1 = year_db.describe()['25%']

yeardb_iqr = yeardb_q3 - yeardb_q1
yeardb_adj = year_db[(year_db < (yeardb_q3 + 1.5 * yeardb_iqr)) & (year_db > (yeardb_q1 - 1.5 * yeardb_iqr))]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_quant(ax, yeardb_adj, xlabel='Years with Database', ylabel='Frequency',
                 title='Distribution of Years with Database', shade=True))

### Manage Staff

In [ ]:
us_cleaned['ManageStaff'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'ManageStaff', us, xlabel='Manage Staff?', ylabel='Frequency', 
              title='Number of People Managing Staff'))

### Years With This Type of Job

In [ ]:
us_cleaned['YearsWithThisTypeOfJob'].value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_quant(ax, us_cleaned['YearsWithThisTypeOfJob'], xlabel='Years With This Type of Job', ylabel='Frequency',
                        title='Frequency of Years With This Type of Job', shade=True))

### Amount of Others on Your Team

Label Encode ordinality

In [ ]:
us_cleaned['OtherPeopleOnYourTeam'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

(plot_cat(ax, 'OtherPeopleOnYourTeam', us_cleaned, xlabel='Number of People on the Team', ylabel='Frequency',
        title='Frequency of Number of Teammates', order=us['OtherPeopleOnYourTeam'].value_counts().sort_index().index))



### Education Level

Label Encode ordinality

In [ ]:
us_cleaned['Education'].value_counts()

In [ ]:
us_cleaned.loc[:, 'Education'] = us['Education'].str.replace('\\(.*\\)', '', regex=True).str.strip()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'Education', us_cleaned, xlabel='Level of Degree', ylabel='Frequency', 
         title='Frequency of Degrees', order=['None', 'Associates', 'Bachelors', 'Masters', 'Doctorate/PhD'])
)

### Hours Worked Per Week

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
(plot_quant(ax, us_cleaned['HoursWorkedPerWeek'], xlabel='Hours Work Per Week', ylabel='Frequency',
                 title='Distribution of Hours Work Per Week', shade=True))

### Telecommute Days Per Week

Label Encode ordinality

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'TelecommuteDaysPerWeek', us_cleaned, xlabel='Num of Days Per Week', ylabel='Count',
             title='Count of Num of Telecommute Days Per Week', 
             order=us_cleaned['TelecommuteDaysPerWeek'].value_counts().sort_index().index))

_ = ax.set_xticklabels(us_cleaned['TelecommuteDaysPerWeek'].value_counts().sort_index().index, rotation=90)



### Employment Sector

Maybe OHE

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'EmploymentSector', us_cleaned, xlabel='Employment Sector', ylabel='Count',
         title='Count of Num of Employment Sector', order=us_cleaned['EmploymentSector'].value_counts().index))

_ = ax.set_xticklabels(us_cleaned['EmploymentSector'].value_counts().index, rotation=90)

### Gender

Might have to drop gender due to high number of third category/random answers/Not Asked.

In [ ]:
def clean_gender(row):
    if (row not in ['Male', 'Female', 'Not Asked']):
        return 'Third Party'
    return row

In [ ]:
us_cleaned.loc[:, 'Gender'] = us['Gender'].apply(clean_gender)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'Gender', us_cleaned, xlabel='Gender', ylabel='Count',
         title='Count of Num of Gender', order=us_cleaned['Gender'].value_counts().sort_index().index))

_ = ax.set_xticklabels(us_cleaned['Gender'].value_counts().sort_index().index, rotation=90)

### Amount of Other Job Duties

Might have to exclude, due to high number of nans/No answers

In [ ]:
us['OtherJobDuties'].replace('Not Asked', np.nan).isna().mean()

In [ ]:
def transform_ojd(row):
    if (row == 'Not Asked'):
        return ['N/A']
    return row.split(',')

def count_ojd(row):
    if ('N/A' in row):
        return np.nan
    elif ('None' in row):
        return 0
    return len(row)

In [ ]:
us_cleaned['OJD_count'] = (us['OtherJobDuties'].replace(np.nan, 'None')
                                               .str.strip()
                                               .replace('\\(.*\\)', '', regex=True)
                                               .apply(transform_ojd)
                                               .apply(count_ojd))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_cat(ax, 'OJD_count', us_cleaned, xlabel='Number of Other Job Duties', ylabel='Frequency', 
                        title='Frequency of Other Job Duties'))

In [ ]:
features + ['OJD_count']

### Feature Engineering

In [ ]:
X = us.loc[:, features + target]

In [ ]:
X.head()

### SalaryUSD

Cleaning erroneous values

In [ ]:
X = X.drop(X[X['SalaryUSD'] < 100].index)

### YearsWithThisDatabase

Cleaning erroneous values

In [ ]:
X[X['YearsWithThisDatabase'] > 100]

In [ ]:
X = X.drop(X[X['YearsWithThisDatabase'] > 100].index)

### ManageStaff

Change it into a categorical variable

In [ ]:
def transform_staff(row):
    return 0 if row == 'No' else 1

In [ ]:
X.loc[:, 'ManageStaff'] = X['ManageStaff'].apply(transform_staff)

In [ ]:
X.head()

### Other People On Your Team

In [ ]:
def transform_team(row):
    if (row == 'None'):
        return 0
    elif (row == 'More than 5'):
        return 6
    else:
        return int(row)

In [ ]:
X.loc[:, 'OtherPeopleOnYourTeam'] = X['OtherPeopleOnYourTeam'].apply(transform_team)

### Education

In [ ]:
def transform_education(row):
    if (row == 'None'):
        return 0
    elif (row == 'Associates'):
        return 1
    elif (row == 'Bachelors'):
        return 2
    elif (row == 'Masters'):
        return 3
    else:
        return 4

In [ ]:
X.loc[:, 'Education'] = X['Education'].apply(transform_education)

### TelecommuteDaysPerWeek

In [ ]:
def transform_telecommute(row):
    if ('None' in row):
        return 0
    elif (row == '5 or more'):
        return 5
    else:
        return int(row)

In [ ]:
X.loc[:, 'TelecommuteDaysPerWeek'] = X['TelecommuteDaysPerWeek'].apply(transform_telecommute)

### EmploymentSector

In [ ]:
# Get dummies later

# X = pd.concat([X, pd.get_dummies(X['EmploymentSector'])], axis=1)
# X = X.drop('EmploymentSector', axis=1)

### LookingForAnotherJob

In [ ]:
def transform_another_job(row):
    if (row == 'No'):
        return 0
    elif (row == 'Yes, but only passively (just curious)'):
        return 1
    else:
        return 2

In [ ]:
X.loc[:, 'LookingForAnotherJob'] = X['LookingForAnotherJob'].apply(transform_another_job)

In [ ]:
X.info()

In [ ]:
X.head()

### More plotting for variables

In [ ]:
sl = pd.concat([X['SalaryUSD'], X['LookingForAnotherJob']], axis=1)

In [ ]:
sl_q3 = sl['SalaryUSD'].describe()['75%']
sl_q1 = sl['SalaryUSD'].describe()['25%']

sl_iqr = sl_q3 - sl_q1
sl = sl.loc[sl.loc[((sl['SalaryUSD'] < (sl_q3 + 1.5 * sl_iqr)) & (sl['SalaryUSD'] > (sl_q1 - 1.5 * sl_iqr))), 'SalaryUSD'].index]



In [ ]:
sns.violinplot(x='LookingForAnotherJob', y='SalaryUSD', data=sl)

In [ ]:
sns.violinplot(x='LookingForAnotherJob', y='HoursWorkedPerWeek', data=X)

**Splitting into 3 groups**

In [ ]:
not_looking = X[X['LookingForAnotherJob'] == 0]
passively_looking = X[X['LookingForAnotherJob'] == 1]
actively_looking = X[X['LookingForAnotherJob'] == 2]

### Testing, with a p value of 0.05

In [ ]:
import scipy.stats as stats

#### Salary difference

In [ ]:
nl_salary = not_looking['SalaryUSD']
pl_salary = passively_looking['SalaryUSD']
al_salary = actively_looking['SalaryUSD']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_kde(ax, nl_salary, pl_salary, al_salary, xlabel='Salary', ylabel='Frequency',
            title='KDE of Salary'))

In [ ]:
nl_q1 = nl_salary.quantile(.25)
nl_q3 = nl_salary.quantile(.75)
nl_iqr = nl_q3 - nl_q1

nl_mask = ((nl_salary <= nl_q3 + (1.5 * nl_iqr)) & (nl_salary >= nl_q1 - (1.5 * nl_iqr)))

pl_q1 = pl_salary.quantile(.25)
pl_q3 = pl_salary.quantile(.75)
pl_iqr = pl_q3 - pl_q1

pl_mask = ((pl_salary <= pl_q3 + (1.5 * pl_iqr)) & (pl_salary >= pl_q1 - (1.5 * pl_iqr)))

al_q1 = al_salary.quantile(.25)
al_q3 = al_salary.quantile(.75)
al_iqr = al_q3 - al_q1

al_mask = ((al_salary <= al_q3 + (1.5 * al_iqr)) & (al_salary >= al_q1 - (1.5 * al_iqr)))



In [ ]:
nl_adj = nl_salary[nl_mask]
pl_adj = pl_salary[pl_mask]
al_adj = al_salary[al_mask]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_kde(ax, nl_adj, pl_adj, al_adj, xlabel='Salary Adj.', ylabel='Frequency',
            title='KDE of Salary Adj'))

In [ ]:
samps_ttest(not_looking['SalaryUSD'], passively_looking['SalaryUSD'], actively_looking['SalaryUSD'])

In [ ]:
samps_ttest(nl_adj, pl_adj, al_adj)

In [ ]:
# Not usable due to unequal sample size of actively looking compared to not and passively looking
# stats.f_oneway(nl_adj, pl_adj, al_adj)

**Years With This Database**

In [ ]:
nl_yrdb = not_looking['YearsWithThisDatabase']
pl_yrdb = passively_looking['YearsWithThisDatabase']
al_yrdb = actively_looking['YearsWithThisDatabase']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_kde(ax, nl_yrdb, pl_yrdb, al_yrdb, xlabel='Years With This Database', ylabel='Frequency',
            title='KDE of Years With This Database'))

In [ ]:
samps_mwu(nl_yrdb, pl_yrdb, al_yrdb)

**Manage Staff difference**

In [ ]:
nl_ms = not_looking['ManageStaff']
pl_ms = passively_looking['ManageStaff']
al_ms = actively_looking['ManageStaff']

In [ ]:
p = samps_chi2(nl_ms, pl_ms, al_ms)

In [ ]:
if 0.05 > p:
    print('Reject null')
else:
    print('Fail to reject null')

**Years With This Type of Job**

In [ ]:
nl_yrjob = not_looking['YearsWithThisTypeOfJob']
pl_yrjob = passively_looking['YearsWithThisTypeOfJob']
al_yrjob = actively_looking['YearsWithThisTypeOfJob']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_kde(ax, nl_yrjob, pl_yrjob, al_yrjob, xlabel='Years With This Type of Job', ylabel='Frequency',
            title='KDE of Years With This Type of Job'))

In [ ]:
samps_mwu(nl_yrjob, pl_yrjob, al_yrjob)

**Other People On Your Team difference**

In [ ]:
nl_people = not_looking['OtherPeopleOnYourTeam']
pl_people = passively_looking['OtherPeopleOnYourTeam']
al_people = actively_looking['OtherPeopleOnYourTeam']

In [ ]:
p = samps_chi2(nl_people, pl_people, al_people)

In [ ]:
if 0.05 > p:
    print('Reject null')
else:
    print('Fail to reject null')

**Education difference**

In [ ]:
nl_edu = not_looking['Education']
pl_edu = passively_looking['Education']
al_edu = actively_looking['Education']

In [ ]:
p = samps_chi2(nl_edu, pl_edu, al_edu)

In [ ]:
if 0.05 > p:
    print('Reject null')
else:
    print('Fail to reject null')

**Hours of Work Per Week difference**

In [ ]:
nl_hours = not_looking['HoursWorkedPerWeek']
pl_hours = passively_looking['HoursWorkedPerWeek']
al_hours = actively_looking['HoursWorkedPerWeek']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
(plot_kde(ax, nl_hours, pl_hours, al_hours, xlabel='Number of Hours', ylabel='Frequency',
            title='KDE of Hours Worked Per Week'))

In [ ]:
stats.f_oneway(nl_hours, pl_hours, al_hours)

In [ ]:
samps_mwu(nl_hours, pl_hours, al_hours)

**TelecommuteDaysPerWeek**

In [ ]:
nl_tele = not_looking['TelecommuteDaysPerWeek']
pl_tele = passively_looking['TelecommuteDaysPerWeek']
al_tele = actively_looking['TelecommuteDaysPerWeek']

In [ ]:
p = samps_chi2(nl_tele, pl_tele, al_tele)

In [ ]:
if 0.05 > p:
    print('Reject null')
else:
    print('Fail to reject null')

**EmploymentSector**

In [ ]:
nl_emp = not_looking['EmploymentSector']
pl_emp = passively_looking['EmploymentSector']
al_emp = actively_looking['EmploymentSector']

In [ ]:
p = samps_chi2(nl_emp, pl_emp, al_emp)

In [ ]:
if 0.05 > p:
    print('Reject null')
else:
    print('Fail to reject null')

### Future plans

In [ ]:
# countries_more_than_ten = countries.index[countries > 10]
# df = df[df['Country'].isin(countries_more_than_ten)]
# df.head()

### Predictive pipelines